In [1]:
!unzip -q /content/negative.zip -d /content/lfw-dataset



In [2]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Flatten, Dense, Layer
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  print(tf.config.experimental.set_memory_growth(gpu,True))
for gpu in gpus:
  print(gpu)
POS_PATH=os.path.join('data','positive')
NEG_PATH=os.path.join('data','negative')
ANC_PATH=os.path.join('data','anchor')



In [3]:
# Create directories if they do not exist
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

print("Directories created successfully!")


Directories created successfully!


In [4]:
import os
for directory in os.listdir('/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'):
    for file in os.listdir(os.path.join('/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled', directory)):
        EX_PATH = os.path.join('/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)
        # Debugging: Print paths to verify
        # print(f"EX_PATH: {EX_PATH}")
        # print(f"NEW_PATH: {NEW_PATH}")


In [5]:
!pip install icrawler

In [7]:
!pip install icrawler Pillow mtcnn

import os  # Added import
from icrawler.builtin import BingImageCrawler
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from mtcnn import MTCNN
import numpy as np
import shutil
import uuid

def scrape_and_store_images(person_name, num_images, positive_path, anchor_path):
    # Ensure the save paths exist
    if not os.path.exists(positive_path):
        os.makedirs(positive_path)
        print(f"Created directory: {positive_path}")
    else:
        print(f"Directory already exists: {positive_path}")

    if not os.path.exists(anchor_path):
        os.makedirs(anchor_path)
        print(f"Created directory: {anchor_path}")
    else:
        print(f"Directory already exists: {anchor_path}")

    # Initialize the Bing crawler
    crawler = BingImageCrawler(storage={"root_dir": "temp_images"})
    print(f"Starting image crawl for '{person_name}' with max_num={num_images}")
    crawler.crawl(keyword=person_name, max_num=num_images)  # Download specified number of images
    print("Image crawling completed.")

    # Process each downloaded image
    image_files = [f for f in os.listdir("temp_images") if f.endswith(('jpg', 'jpeg', 'png'))]
    print(f"Number of images downloaded: {len(image_files)}")

    if not image_files:
        raise FileNotFoundError("No images were downloaded. Please refine your search query.")

    detector = MTCNN()
    saved_count = 0
    for idx, image_file in enumerate(image_files):
        image_path = os.path.join("temp_images", image_file)
        try:
            downloaded_image = Image.open(image_path).convert("RGB")  # Ensure the image is in RGB mode
            image_np = np.array(downloaded_image)  # Convert to NumPy array for MTCNN

            # Detect faces
            faces = detector.detect_faces(image_np)
            if faces:
                # Get the first detected face's bounding box
                x, y, width, height = faces[0]["box"]

                # Expand the bounding box by 30%
                expansion_factor = 0.3
                x_expand = int(width * expansion_factor)
                y_expand = int(height * expansion_factor)

                # Calculate the new bounding box coordinates
                x_new = max(0, x - x_expand)
                y_new = max(0, y - y_expand)
                width_new = min(downloaded_image.width, x + width + x_expand) - x_new
                height_new = min(downloaded_image.height, y + height + y_expand) - y_new

                # Crop the expanded face region
                cropped_face = downloaded_image.crop((x_new, y_new, x_new + width_new, y_new + height_new))

                # Pad to maintain aspect ratio
                max_side = max(cropped_face.size)
                padded_face = ImageOps.expand(
                    cropped_face,
                    border=(max_side - cropped_face.width) // 2,
                    fill=(0, 0, 0)
                )

                # Resize to 250x250
                resized_face = padded_face.resize((250, 250))

                # Save the processed image with a unique name
                save_path = positive_path if idx < num_images // 2 else anchor_path  # Split into positive and anchor
                filename = f"{uuid.uuid4().hex}.jpg"
                save_path_full = os.path.join(save_path, filename)
                resized_face.save(save_path_full)

                print(f"Saved: {save_path_full}")
                saved_count += 1

            else:
                print(f"No face detected in {image_file}. Skipping.")

        except Exception as e:
            print(f"Error processing {image_file}: {e}")

    print(f"Total images saved: {saved_count}")

    # Clean up temporary folder
    shutil.rmtree("temp_images")
    print("Temporary images folder removed.")

# Example usage
POS_PATH = os.path.join('data', 'positive')
ANCHOR_PATH = os.path.join('data', 'anchor')
scrape_and_store_images(
    person_name="Elon Musk",
    num_images=800,
    positive_path=POS_PATH,
    anchor_path=ANC_PATH  # Corrected variable name
)


Directory already exists: data/positive
Directory already exists: data/anchor
Starting image crawl for 'Elon Musk' with max_num=800


ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining r

Image crawling completed.
Number of images downloaded: 287
Saved: data/positive/b30d5984cc2c4f74a9b1b9f31e2124a4.jpg
Saved: data/positive/41d04f1c6d3a4e0f97fa24b81880bbd7.jpg
No face detected in 000274.jpg. Skipping.
Saved: data/positive/2ee5cf7d2bbd42b99e553d5cb0e78e40.jpg
Saved: data/positive/f90dc439357d47de85e38a4c1e16a531.jpg
Saved: data/positive/7702ef7f4de540cbbcc4cac2a9ba8188.jpg
Saved: data/positive/1e7dafb58629453f971d397a271be9a8.jpg
Saved: data/positive/9a0393fcb8c444f8aeb2a607d60b9289.jpg
Saved: data/positive/73f9beea1f5d4aefbc9c11178d8627fb.jpg
Saved: data/positive/76be7006b21246e78cf20b5bd9a658fb.jpg
Saved: data/positive/0efb792b99454cdcb45a8ba922ff3b9a.jpg
Saved: data/positive/512f162a96084af58b660e3d5bc2127e.jpg
Saved: data/positive/8c3d617e7546424db4eecbd7258a1f40.jpg
Saved: data/positive/085833a858e54676b5e2ec64413314f9.jpg
No face detected in 000284.jpg. Skipping.
Saved: data/positive/5c28375677134442824b2ad010604be9.jpg
Saved: data/positive/c77e669998d34e458eab52e8

In [8]:
# Install necessary packages
!pip install icrawler Pillow mtcnn

# Import required modules
import os
from icrawler.builtin import BingImageCrawler
from PIL import Image, ImageOps
from mtcnn import MTCNN
import numpy as np
import shutil
import uuid

def scrape_and_store_anchor_images(person_name, num_images, anchor_path):
    """
    Scrapes images of a specified person and saves all processed images to the anchor path.

    Parameters:
    - person_name (str): The name of the person to search for.
    - num_images (int): The number of images to download.
    - anchor_path (str): The directory path where anchor images will be saved.
    """

    # Ensure the anchor directory exists
    if not os.path.exists(anchor_path):
        os.makedirs(anchor_path)
        print(f"Created directory: {anchor_path}")
    else:
        print(f"Directory already exists: {anchor_path}")

    # Initialize the Bing crawler to download images
    crawler = BingImageCrawler(storage={"root_dir": "temp_images"})
    print(f"Starting image crawl for '{person_name}' with max_num={num_images}")
    crawler.crawl(keyword=person_name, max_num=num_images)
    print("Image crawling completed.")

    # List all downloaded image files
    image_files = [f for f in os.listdir("temp_images") if f.lower().endswith(('jpg', 'jpeg', 'png'))]
    print(f"Number of images downloaded: {len(image_files)}")

    if not image_files:
        raise FileNotFoundError("No images were downloaded. Please refine your search query.")

    # Initialize MTCNN for face detection
    detector = MTCNN()
    saved_count = 0

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join("temp_images", image_file)
        try:
            # Open and convert image to RGB
            downloaded_image = Image.open(image_path).convert("RGB")
            image_np = np.array(downloaded_image)

            # Detect faces in the image
            faces = detector.detect_faces(image_np)
            if faces:
                # Get the first detected face's bounding box
                x, y, width, height = faces[0]["box"]

                # Expand the bounding box by 30%
                expansion_factor = 0.3
                x_expand = int(width * expansion_factor)
                y_expand = int(height * expansion_factor)

                # Calculate new bounding box coordinates with expansion
                x_new = max(0, x - x_expand)
                y_new = max(0, y - y_expand)
                width_new = min(downloaded_image.width, x + width + x_expand) - x_new
                height_new = min(downloaded_image.height, y + height + y_expand) - y_new

                # Crop the expanded face region
                cropped_face = downloaded_image.crop((x_new, y_new, x_new + width_new, y_new + height_new))

                # Pad to maintain aspect ratio
                max_side = max(cropped_face.size)
                padded_face = ImageOps.expand(
                    cropped_face,
                    border=(max_side - cropped_face.width) // 2,
                    fill=(0, 0, 0)
                )

                # Resize to 250x250 pixels
                resized_face = padded_face.resize((250, 250))

                # Generate a unique filename and save the image to the anchor path
                filename = f"{uuid.uuid4().hex}.jpg"
                save_path_full = os.path.join(anchor_path, filename)
                resized_face.save(save_path_full)

                # print(f"Saved: {save_path_full}")
                saved_count += 1
            else:
                print(f"No face detected in {image_file}. Skipping.")

        except Exception as e:
            print(f"Error processing {image_file}: {e}")

    print(f"Total images saved to anchor: {saved_count}")

    # Clean up temporary images
    shutil.rmtree("temp_images")
    print("Temporary images folder removed.")

# Example usage
ANCHOR_PATH = os.path.join('data', 'anchor')
scrape_and_store_anchor_images(
    person_name="Elon Musk",
    num_images=400,  # Adjust as needed
    anchor_path=ANCHOR_PATH
)


Directory already exists: data/anchor
Starting image crawl for 'Elon Musk' with max_num=400


ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/7GZY2GHEDQI6XCGFJ7LDQLCHZM.jpg, error: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5), remaining r

Image crawling completed.
Number of images downloaded: 244
No face detected in 000143.jpg. Skipping.
Total images saved to anchor: 243
Temporary images folder removed.


In [9]:
def preprocess(file_path):
    # Read the file
    byte_img = tf.io.read_file(file_path)
    # Determine file extension and decode accordingly

    img = tf.io.decode_jpeg(byte_img, channels=3)

    img = tf.image.resize(img, (100, 100))
    img = img / 255.0

    return img


In [10]:
import os

# Check if directories exist
print("Anchor directory exists:", os.path.exists(ANC_PATH))
print("Anchor directory contents:", os.listdir(ANC_PATH) if os.path.exists(ANC_PATH) else "Directory not found")


Anchor directory exists: True
Anchor directory contents: ['fcc15037fefc48d2bc7629424113783c.jpg', '8ab1da0e740843cd9873d05deeb3d3dd.jpg', 'f2021f40bd554592ba3361fa029f6304.jpg', '4a929d77957742a2a2463560f1311c1f.jpg', 'a1f215ab19314cde97352ae6db55200e.jpg', '2de94ba2acf94e5ba8f340d61e28ee44.jpg', '92c4cab5d7924d929ecd21862277e19e.jpg', 'e0bdfc6511794425937c67a68e186f08.jpg', 'f655c90e8ea3428294fb4055aa73f0d3.jpg', '7b9ba20548884e0b903270ed41f54ac6.jpg', '797d6da339d3457889ef698c6514758a.jpg', '7409517586cf495dbfcb81ca3059b6ec.jpg', 'b99c00fd9a9049939e50d74684de56d9.jpg', 'b2056fe2f90249b98f4926e6dd5f6a75.jpg', '30dbd0430b0548429cc28ee944d6a71b.jpg', 'd6971343a6d84ddba492d394468e59fb.jpg', 'a9fd95f9830341e2badb6717f59bab8d.jpg', '646e6080a49d43e3ae9c926bf8059b9d.jpg', '5ce4ab3195d74696b93d45fe35d7b13e.jpg', 'e5b5fdaed3c94604803a858900e2d1dc.jpg', '5e9d8fd6f018447fbb2b5888ed6f3fab.jpg', 'e44a7a5b275a4e6f9a96918c4bffc443.jpg', '234525fa146b46e0acb612bee74e53ff.jpg', 'de2dcf8855a24a62be848

In [11]:
negative = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg', shuffle=False).take(300)
positive = tf.data.Dataset.list_files(POS_PATH + '/*.jpg', shuffle=False).take(300)
anchor = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg', shuffle=True).take(300)
# positive = tf.data.Dataset.list_files(POS_PATH + '/*.{jpg,jpeg,png}', shuffle=False).take(300)



In [12]:
positives=tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives=tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data=positives.concatenate(negatives)

In [13]:
def preprocess_twin(input_img, validation_img, label):
  return (preprocess(input_img), preprocess(validation_img), label)

In [14]:
#data pipeline
data=data.map(preprocess_twin)
data=data.cache()
data=data.shuffle(buffer_size=64)


In [15]:
train_data=data.take(round(len(data)*.7))
train_data=train_data.batch(16)
train_data=train_data.prefetch(8)

test_data=data.skip(round(len(data)*.7))
test_data=test_data.take(round(len(data)*.3))
test_data=test_data.batch(16)
test_data=test_data.prefetch(8)


In [16]:
inp=Input(shape=(100,100,3), name='inpu_image')
c1=Conv2D(64,(10,10), activation='relu')(inp)
m1=MaxPooling2D(64,(2,2), padding='same')(c1)
c2=Conv2D(128,(7,7), activation='relu')(m1)
m2=MaxPooling2D(64,(2,2),padding='same')(c2)
c3=Conv2D(128,(4,4),activation='relu')(m2)
m3=MaxPooling2D(64,(2,2),padding='same')(c3)
c4=Conv2D(256,(4,4), activation='relu')(m3)
f1=Flatten()(c4)
d1=Dense(4096, activation='sigmoid')(f1)


In [22]:
mod=Model(inputs=[inp],outputs=[d1], name='embedding')

In [23]:
mod.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inpu_image (InputLayer)              │ (None, 100, 100, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 91, 91, 64)          │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 46, 46, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 40, 40, 128)         │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 17, 17, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 9, 9, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 6, 6, 256)           │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │      37,752,832 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
def make_embedding():
    inp = Input(shape=(100,100,3), name='input_image')

    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    # Third block
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)


    return Model(inputs=inp, outputs=d1, name='embedding')

In [25]:
embedding = make_embedding()

In [26]:
embedding.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)             │ (None, 100, 100, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 91, 91, 64)          │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 46, 46, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 40, 40, 128)         │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 17, 17, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 9, 9, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 6, 6, 256)           │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │      37,752,832 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Siamese L1 Distance class
class L1Dist(Layer):

    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()

    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)


In [28]:
l1 = L1Dist()
l1

<L1Dist name=l1_dist, built=False>

In [29]:
input_image=Input(name='input_img', shape=(100,100,3))
print(embedding(input_image))
validation_image=Input(name='validation_img',shape=(100,100,3))
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)

<KerasTensor shape=(None, 4096), dtype=float32, sparse=False, name=keras_tensor_18>


In [30]:
siamese_layer=L1Dist()
siamese_layer(inp_embedding, val_embedding)

<KerasTensor shape=(None, 4096), dtype=float32, sparse=False, name=keras_tensor_21>

In [31]:
def make_siamese_model():

    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))

    # Validation image in the network
    validation_image = Input(name='validation_img', shape=(100,100,3))

    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    # # Classification layer
    classifier = Dense(1, activation='sigmoid')(distances)

    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [32]:
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_img (InputLayer)    │ (None, 100, 100, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ validation_img            │ (None, 100, 100, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Functional)    │ (None, 4096)           │     38,960,448 │ input_img[0][0],       │
│                           │                        │                │ validation_img[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ l1_dist_2 (L1Dist)        │ (None, 4096)           │              0 │ embedding[3][0],       │
│                           │                        │                │ embedding[4][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │          4,097 │ l1_dist_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
binary_cross_loss=tf.losses.BinaryCrossentropy()
opt=tf.keras.optimizers.Adam(1e-4)

checkpoint_dir='./training_checkpoints'
checkpoint_prefix=os.path.join(checkpoint_dir,'ckpt')
checkpoint=tf.train.Checkpoint(opt=opt,siamese_model=siamese_model)

In [1]:
@tf.function
def train_step(batch):
  #record all of opetarions
  with tf.GradientTape() as tape:
    X=batch[:2]
    y=batch[2]

    yhat=siamese_model(X,training=True)
    loss=binary_cross_loss(y,yhat)

  grad=tape.gradient(loss, siamese_model.trainable_variables)

  opt.apply_gradients(zip(grad,siamese_model.trainable_variables))

  return loss


NameError: name 'tf' is not defined

In [39]:
def train(data,EPOCHS):
  #loop through epochs
  for epoch in range(1, EPOCHS+1):
    print('\n Epoch {} {}'.format(epoch, EPOCHS))
    progbar=tf.keras.utils.Progbar(len(train_data))

    for idx, batch in enumerate(train_data):
      train_step(batch)
      progbar.update(idx+1)
    if epoch % 10 ==0:
      checkpoint.save(file_prefix=checkpoint_prefix)


In [ ]:
EPOCHS=50
train(train_data,EPOCHS)


 Epoch 1 50
 1/22 ━━━━━━━━━━━━━━━━━━━━ 14:46 42s/step